In [25]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np


In [26]:
PUZZLE_DIM = 3

action = namedtuple('Action', ['pos1', 'pos2']) # pos1, pos2 are tuples of (x, y). pos1 is current position, pos2 is target position

#Returns list of all available actions for "0" in the given state
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions


In [27]:
# Returns new state after performing the given action on the given state
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


RANDOMIZE_STEPS = 500
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal_state = state.copy()
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 500/500 [00:00<00:00, 293103.00it/s]


array([[0, 5, 8],
       [3, 1, 7],
       [6, 4, 2]])

In [28]:
# The Manhattan Distance measures the total distance each tile is from its goal position
def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    dist = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value == 0:
                continue
            goal_pos = np.argwhere(goal == value)[0]
            dist += abs(x - goal_pos[0]) + abs(y - goal_pos[1])
    return dist

# The linear conflict heuristic extends the Manhattan Distance by adding 2 for each pair of tiles that are in their goal row/column but are in the wrong order
def linear_conflict(state: np.ndarray, goal: np.ndarray) -> int:
    md = manhattan_distance(state, goal)  # Use the previously defined Manhattan Distance
    conflict = 0
    size = state.shape[0]

    for row in range(size):
        goal_row = goal[row]
        current_row = state[row]

        # Count conflicts in the current row
        for i in range(size):
            for j in range(i + 1, size):
                if current_row[i] in goal_row and current_row[j] in goal_row:
                    if goal_row.tolist().index(current_row[i]) > goal_row.tolist().index(current_row[j]):
                        conflict += 2  # Each conflict adds 2 to the heuristic

    for col in range(size):
        goal_col = goal[:, col]
        current_col = state[:, col]

        # Count conflicts in the current column
        for i in range(size):
            for j in range(i + 1, size):
                if current_col[i] in goal_col and current_col[j] in goal_col:
                    if goal_col.tolist().index(current_col[i]) > goal_col.tolist().index(current_col[j]):
                        conflict += 2  # Each conflict adds 2 to the heuristic

    return md + conflict


In [29]:
from heapq import heappop, heappush


def solve_puzzle(start: np.ndarray, goal: np.ndarray) -> list[np.ndarray]:
    start_tuple = tuple(start.flatten())
    goal_tuple = tuple(goal.flatten())

    open_set = []
    heappush(open_set, (0, start_tuple))

    came_from = {}
    g_score = {start_tuple: 0}
    f_score = {start_tuple: linear_conflict(start, goal)}

    actions_evaluated = 0 #Used for evaluating cost

    while open_set:
        _, current = heappop(open_set)
        current_state = np.array(current).reshape((PUZZLE_DIM, PUZZLE_DIM))

        if current == goal_tuple:
            path = []
            while current:
                path.append(np.array(current).reshape((PUZZLE_DIM, PUZZLE_DIM)))
                current = came_from.get(current)
            return path[::-1], actions_evaluated

        for action in available_actions(current_state):
            actions_evaluated += 1
            neighbor_state = do_action(current_state, action)
            neighbor_tuple = tuple(neighbor_state.flatten())
            tentative_g_score = g_score[current] + 1

            if tentative_g_score < g_score.get(neighbor_tuple, float('inf')):
                came_from[neighbor_tuple] = current
                g_score[neighbor_tuple] = tentative_g_score
                f_score[neighbor_tuple] = tentative_g_score + linear_conflict(neighbor_state, goal)
                heappush(open_set, (f_score[neighbor_tuple], neighbor_tuple))

    return None, actions_evaluated


In [30]:
solution, cost = solve_puzzle(state, goal_state)

if solution:
    for step, s in enumerate(solution):
        print(f"Step {step}:")
        print(s)
        
    quality = len(solution)-1
    print(f"Total actions in solution (quality): {quality}")
    print(f"Total actions evaluated (cost): {cost}")
    
else:
    print("No solution found.")
    print(f"Total actions evaluated (cost): {cost}")


Step 0:
[[0 5 8]
 [3 1 7]
 [6 4 2]]
Step 1:
[[3 5 8]
 [0 1 7]
 [6 4 2]]
Step 2:
[[3 5 8]
 [1 0 7]
 [6 4 2]]
Step 3:
[[3 5 8]
 [1 7 0]
 [6 4 2]]
Step 4:
[[3 5 0]
 [1 7 8]
 [6 4 2]]
Step 5:
[[3 0 5]
 [1 7 8]
 [6 4 2]]
Step 6:
[[0 3 5]
 [1 7 8]
 [6 4 2]]
Step 7:
[[1 3 5]
 [0 7 8]
 [6 4 2]]
Step 8:
[[1 3 5]
 [7 0 8]
 [6 4 2]]
Step 9:
[[1 3 5]
 [7 4 8]
 [6 0 2]]
Step 10:
[[1 3 5]
 [7 4 8]
 [0 6 2]]
Step 11:
[[1 3 5]
 [0 4 8]
 [7 6 2]]
Step 12:
[[1 3 5]
 [4 0 8]
 [7 6 2]]
Step 13:
[[1 3 5]
 [4 8 0]
 [7 6 2]]
Step 14:
[[1 3 5]
 [4 8 2]
 [7 6 0]]
Step 15:
[[1 3 5]
 [4 8 2]
 [7 0 6]]
Step 16:
[[1 3 5]
 [4 0 2]
 [7 8 6]]
Step 17:
[[1 3 5]
 [4 2 0]
 [7 8 6]]
Step 18:
[[1 3 0]
 [4 2 5]
 [7 8 6]]
Step 19:
[[1 0 3]
 [4 2 5]
 [7 8 6]]
Step 20:
[[1 2 3]
 [4 0 5]
 [7 8 6]]
Step 21:
[[1 2 3]
 [4 5 0]
 [7 8 6]]
Step 22:
[[1 2 3]
 [4 5 6]
 [7 8 0]]
Total actions in solution (quality): 22
Total actions evaluated (cost): 385
